In [32]:
import numpy as np
from data import X_train, X_test, y_train, y_test
from random_forest import RandomForestList,PredictRandomForest
from tree import print_tree


In [33]:
random_forest_list = RandomForestList(X_train, y_train, n_trees=1, max_depth=10, min_samples_split=5, min_samples_leaf=0, max_features=30, bootsrap=False)

TypeError: RandomForestList() got an unexpected keyword argument 'bootsrap'

In [ ]:
print(random_forest_list.__len__())

500


In [ ]:
for i in random_forest_list:
    print_tree(i.tree)

Node d=0 samples=455 gini=0.4646 pred=0 feat=6 thr=0.14695
  Node d=1 samples=304 gini=0.1286 pred=0 feat=5 thr=929.8
    Node d=2 samples=289 gini=0.0603 pred=0 feat=11 thr=0.11085
      Node d=3 samples=245 gini=0.0000 pred=0 feat=None thr=None
      Node d=3 samples=44 gini=0.3254 pred=0 feat=4 thr=32.575
        Node d=4 samples=38 gini=0.1454 pred=0 feat=None thr=None
        Node d=4 samples=6 gini=0.0000 pred=1 feat=None thr=None
    Node d=2 samples=15 gini=0.3200 pred=1 feat=1 thr=0.35919999999999996
      Node d=3 samples=4 gini=0.3750 pred=0 feat=None thr=None
      Node d=3 samples=11 gini=0.0000 pred=1 feat=None thr=None
  Node d=1 samples=151 gini=0.0640 pred=1 feat=2 thr=0.13565
    Node d=2 samples=147 gini=0.0135 pred=1 feat=1 thr=0.085885
      Node d=3 samples=7 gini=0.2449 pred=1 feat=None thr=None
      Node d=3 samples=140 gini=0.0000 pred=1 feat=None thr=None
    Node d=2 samples=4 gini=0.0000 pred=0 feat=None thr=None
Node d=0 samples=455 gini=0.4681 pred=0 feat

In [ ]:
y_pred = PredictRandomForest(random_forest_list, X_train)
accuracy = np.mean(y_pred == y_train) 
print("Accuracy (en train):", accuracy)

Accuracy (en train): 0.7626373626373626
